# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [60]:
import pandas as pd
data = pd.read_csv("06_string_data/recipes_sample.csv", sep=',')
data_ = data.loc[:, ['id', 'minutes']]
print("|{: ^14}|{: ^11}|".format(*data_.columns))
print("|" + "-" * 26 + "|")
for row in data_.sample(5).values:
    print(f"|{row[0]: ^14}|{row[1]: ^11}|")

|      id      |  minutes  |
|--------------------------|
|    26598     |    22     |
|    301525    |    75     |
|    487065    |    55     |
|    130300    |    50     |
|    59852     |    15     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
from bs4 import BeautifulSoup
steps_data = BeautifulSoup(open("06_string_data/steps_sample.xml", "r").read(), "xml")

In [47]:

recipe_id_element = None

def show_info(recipe_id=44123, name="", steps=[], minutes=0, author_id=0):
    global recipe_id_element
    recipe = data[data['id'] == recipe_id].to_dict('list')
    recipe_steps = []
    recipe_author = recipe["contributor_id"][0]
    recipe_name = f'"{recipe["name"][0]}"'
    recipe_time = recipe["minutes"][0]

    recipe_id_element = steps_data.find('id', text=recipe_id)
    recipe_steps = [ e.text for e in recipe_id_element.next_sibling.next_sibling.find_all('step')]

    steps_str = "\n".join(map(lambda x: f"{x[0]}. {x[1].capitalize()}.",enumerate(recipe_steps, 1)))
    return recipe_name + "\n\n" + steps_str + "\n" + 10 * "-" + "\nАвтор: " + str(recipe_author) + '\nСреднее время приготовления: ' + str(recipe_time) + " минут"

print(show_info(170895))


"leeks and parsnips  sauteed or creamed"

1. Clean the leeks and discard the dark green portions.
2. Cut the leeks lengthwise then into one-inch pieces.
3. Melt the butter in a medium skillet , med.
4. Heat.
5. Add the garlic and fry 'til fragrant.
6. Add leeks and fry until the leeks are tender , about 6-minutes.
7. Meanwhile , peel and chunk the parsnips into one-inch pieces.
8. Place in a steaming basket and steam 'til they are as tender as you prefer.
9. I like them fork-tender.
10. Drain parsnips and add to the skillet with the leeks.
11. Add salt and pepper.
12. Gently sautee together for 5-minutes.
13. At this point you can serve it , or continue on and cream it:.
14. In a jar with a screw top , add the half-n-half and arrowroot.
15. Shake 'til blended.
16. Turn heat to low under the leeks and parsnips.
17. Pour in the arrowroot mixture , stirring gently as you pour.
18. If too thick , gradually add the water.
19. Let simmer for a couple of minutes.
20. Taste to adjust seasoning

 ## Работа с регулярными выражениями


3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [50]:
import re
pattern = re.compile(r"([\d]+ (hour[s]*|minute[s]*))")
pattern.findall(show_info(25082))


[('20 minutes', 'minutes'),
 ('10 minutes', 'minutes'),
 ('2 hours', 'hours'),
 ('10 minutes', 'minutes'),
 ('20 minutes', 'minutes'),
 ('30 minutes', 'minutes')]

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [51]:

search_pattern = re.compile(r"^this.*[,] ?but")
descriptions = pd.Series(data.description).dropna()
descriptions_pattern = descriptions.str.match(search_pattern)

print(f"Кол-во рецептов: {descriptions_pattern.sum()}")

Кол-во рецептов: 865


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [52]:
print(re.sub(r' / ', '/', show_info(72367)))

"bavarian apple cheese cake tart"

1. Mix butter , flour , 1/3 c.
2. Sugar and 1-1/4 t.
3. Vanilla.
4. Press into greased 9" springform pan.
5. Mix cream cheese , 1/4 c.
6. Sugar , eggs and 1/2 t.
7. Vanilla beating until fluffy.
8. Pour over dough.
9. Combine apples , 1/3 c.
10. Sugar and cinnamon.
11. Arrange on top of cream cheese mixture and sprinkle with almonds.
12. Bake at 350 for 45-55 minutes , or until tester comes out clean.
----------
Автор: 93006
Среднее время приготовления: 65 минут


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [57]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AndroidS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [58]:
import nltk
recipes = BeautifulSoup(open('06_string_data/steps_sample.xml', 'r').read(), 'xml').find_all('recipe')

unique_words = set()
for word in recipes:
    unique_words.update(nltk.word_tokenize(word.steps.text.strip()))

len(unique_words)

19604

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [61]:
data['description'] = data['description'].fillna("---")
descriptions = data["description"].apply(nltk.sent_tokenize)
data_ = descriptions.to_dict()

In [62]:
data.iloc[sorted(data_, key=lambda x: len(data_[x]), reverse=True)[:5]]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [63]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AndroidS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [64]:
pos_tag_values = nltk.pos_tag(nltk.word_tokenize(data[data['id'] == 241106].name.values[0]))
pos_tag_values

[('eggplant', 'JJ'),
 ('steaks', 'NNS'),
 ('with', 'IN'),
 ('chickpeas', 'NNS'),
 ('feta', 'VBP'),
 ('cheese', 'JJ'),
 ('and', 'CC'),
 ('black', 'JJ'),
 ('olives', 'NNS')]

In [65]:
# Определение максимальной длины для вывода
formatted_values = [(value[-1], value[-2], max(len(value[0]), len(value[1]))) for value in pos_tag_values]
formatted_values

[('JJ', 'eggplant', 8),
 ('NNS', 'steaks', 6),
 ('IN', 'with', 4),
 ('NNS', 'chickpeas', 9),
 ('VBP', 'feta', 4),
 ('JJ', 'cheese', 6),
 ('CC', 'and', 3),
 ('JJ', 'black', 5),
 ('NNS', 'olives', 6)]

In [66]:
print(''.join([f"{ind[0]:^{ind[2] + 1}}" for ind in formatted_values]) + '\n' + ''.join([f"{ind[1]:^{ind[2] + 1}}" for ind in formatted_values]))

   JJ      NNS   IN     NNS     VBP   JJ    CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
